In [1]:
import sys
sys.path.append('..')
from source.code import data_loader

In [2]:
sys.path

['',
 '/home/bender/anaconda2/lib/python27.zip',
 '/home/bender/anaconda2/lib/python2.7',
 '/home/bender/anaconda2/lib/python2.7/plat-linux2',
 '/home/bender/anaconda2/lib/python2.7/lib-tk',
 '/home/bender/anaconda2/lib/python2.7/lib-old',
 '/home/bender/anaconda2/lib/python2.7/lib-dynload',
 '/home/bender/.local/lib/python2.7/site-packages',
 '/home/bender/anaconda2/lib/python2.7/site-packages',
 '/home/bender/anaconda2/lib/python2.7/site-packages/Sphinx-1.5.1-py2.7.egg',
 '/home/bender/anaconda2/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg',
 '/home/bender/anaconda2/lib/python2.7/site-packages/IPython/extensions',
 '/home/bender/.ipython',
 '..']